<a href="https://colab.research.google.com/github/Mileej/predicciones-de-ventas-alimenticias/blob/main/Proyecto1_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
#importar librerias
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_selector, make_column_transformer

In [86]:
df=pd.read_csv('/content/drive/MyDrive/Semestre 2022-2/CODING_DOJO_DS/Machine_Learning/sales_predictions.csv', sep=',')
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


# revisión inicial

In [87]:
df.shape

(8523, 12)

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [89]:
df.nunique()

Item_Identifier              1559
Item_Weight                   415
Item_Fat_Content                5
Item_Visibility              7880
Item_Type                      16
Item_MRP                     5938
Outlet_Identifier              10
Outlet_Establishment_Year       9
Outlet_Size                     3
Outlet_Location_Type            3
Outlet_Type                     4
Item_Outlet_Sales            3493
dtype: int64

Usando el imputer para  manejo de datos faltantes 

In [90]:
df.duplicated().sum() 

0

In [91]:
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [92]:
df['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [93]:
#para este caso reemplacemos LF por Low Fat, reg por Regular y low fat por Low Fat
df['Item_Fat_Content']=df['Item_Fat_Content'].replace('LF','Low Fat')
df['Item_Fat_Content']=df['Item_Fat_Content'].replace('reg','Regular')
df['Item_Fat_Content']=df['Item_Fat_Content'].replace('low fat','Low Fat')
df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [94]:
#ordinal encoder 
item_fat_r={'Regular':1,'Low Fat':2}
df['Item_Fat_Content'].replace(item_fat_r,inplace=True)

In [95]:
df['Outlet_Size'].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [96]:
out_sise_r={'High':3,'Medium':2,'Small':1}
df['Outlet_Size'].replace(out_sise_r,inplace=True)

In [97]:
df['Outlet_Size'].value_counts()

2.0    2793
1.0    2388
3.0     932
Name: Outlet_Size, dtype: int64

In [98]:
df['Outlet_Type'].value_counts()

Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

In [99]:
df['Outlet_Location_Type'].value_counts()

Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64

In [100]:
df.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [101]:
most_common_b=df['Item_Weight'].mode()[0]
most_common_c=df['Outlet_Size'].mode()[0]
df.fillna(most_common_b,inplace=True)
df.fillna(most_common_c, inplace=True)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,2,0.016047,Dairy,249.8092,OUT049,1999,2.00,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,1,0.019278,Soft Drinks,48.2692,OUT018,2009,2.00,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,2,0.016760,Meat,141.6180,OUT049,1999,2.00,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,1,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,12.15,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,2,0.000000,Household,53.8614,OUT013,1987,3.00,Tier 3,Supermarket Type1,994.7052


#Preparación de los datos 

In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8523 non-null   float64
 2   Item_Fat_Content           8523 non-null   int64  
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                8523 non-null   float64
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(5), int64(2), object(5)
memory usage: 799.2+ KB


In [103]:
df=df.drop(columns=['Item_Identifier'])

In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Weight                8523 non-null   float64
 1   Item_Fat_Content           8523 non-null   int64  
 2   Item_Visibility            8523 non-null   float64
 3   Item_Type                  8523 non-null   object 
 4   Item_MRP                   8523 non-null   float64
 5   Outlet_Identifier          8523 non-null   object 
 6   Outlet_Establishment_Year  8523 non-null   int64  
 7   Outlet_Size                8523 non-null   float64
 8   Outlet_Location_Type       8523 non-null   object 
 9   Outlet_Type                8523 non-null   object 
 10  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(5), int64(2), object(4)
memory usage: 732.6+ KB


tranformación de datos 

In [105]:
#variables categoricas nominales --> (One-hot encoding)
Item_Type_dummy=pd.get_dummies(df['Item_Type'], prefix='Item_Type')
Outlet_Identifier_dummy=pd.get_dummies(df['Outlet_Identifier'], prefix='Outlet_Identifier')
Outlet_Location_Type_dummy=pd.get_dummies(df['Outlet_Location_Type'],prefix='Outlet_Location_Type')
Outlet_Type_dummy=pd.get_dummies(df['Outlet_Type'], prefix='Outlet_Type')

In [106]:
#unir las variables 
df=pd.concat([df,Item_Type_dummy],axis=1)
df=pd.concat([df,Outlet_Identifier_dummy],axis=1)
df=pd.concat([df,Outlet_Location_Type_dummy],axis=1)
df=pd.concat([df,Outlet_Type_dummy],axis=1)

In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 44 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Item_Weight                      8523 non-null   float64
 1   Item_Fat_Content                 8523 non-null   int64  
 2   Item_Visibility                  8523 non-null   float64
 3   Item_Type                        8523 non-null   object 
 4   Item_MRP                         8523 non-null   float64
 5   Outlet_Identifier                8523 non-null   object 
 6   Outlet_Establishment_Year        8523 non-null   int64  
 7   Outlet_Size                      8523 non-null   float64
 8   Outlet_Location_Type             8523 non-null   object 
 9   Outlet_Type                      8523 non-null   object 
 10  Item_Outlet_Sales                8523 non-null   float64
 11  Item_Type_Baking Goods           8523 non-null   uint8  
 12  Item_Type_Breads    

In [108]:
df=df.drop(columns=['Outlet_Type','Outlet_Location_Type','Item_Type','Outlet_Identifier'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 40 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Item_Weight                      8523 non-null   float64
 1   Item_Fat_Content                 8523 non-null   int64  
 2   Item_Visibility                  8523 non-null   float64
 3   Item_MRP                         8523 non-null   float64
 4   Outlet_Establishment_Year        8523 non-null   int64  
 5   Outlet_Size                      8523 non-null   float64
 6   Item_Outlet_Sales                8523 non-null   float64
 7   Item_Type_Baking Goods           8523 non-null   uint8  
 8   Item_Type_Breads                 8523 non-null   uint8  
 9   Item_Type_Breakfast              8523 non-null   uint8  
 10  Item_Type_Canned                 8523 non-null   uint8  
 11  Item_Type_Dairy                  8523 non-null   uint8  
 12  Item_Type_Frozen Foo

In [109]:
#Dividir las columnas de datos en X (características) e y (objetivo)
y=df['Item_Outlet_Sales']
X= df.drop(columns='Item_Outlet_Sales')

In [110]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import set_config


In [111]:
# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [112]:
# instancien el escalador
scaler = StandardScaler()
# ajusten el escalador en los datos de entrenamiento
scaler.fit(X_train)

StandardScaler()

# Modelos de regresión

1) modelo de regresión lineal para predecir las ventas.

* Creen un modelo de regresión lineal.
* Evalúen el rendimiento del modelo basándose en r^2.
* Evalúen el rendimiento del modelo basándose en RECM.

In [113]:
from sklearn.linear_model import LinearRegression

In [114]:
# Make a linear regression instance
reg = LinearRegression()

reg.fit(X_train,y_train)

LinearRegression()

✈ modelo basándose en r^2.

In [115]:
train_score = reg.score(X_train, y_train)
print(train_score)

0.5615551419174629


In [116]:
test_score = reg.score(X_test, y_test)
print(test_score)

0.5671041872384911


La predicción es muy baja, sin embargo no hay un sobre ajuste 

In [117]:
# Obtengan las predicciones del conjunto de entrenamiento
train_preds = reg.predict(X_train)
# Obtengan las predicciones del conjunto de prueba
test_preds = reg.predict(X_test)
test_preds

array([1360.55910613,  717.47568991,  881.55087906, ..., 3303.68786496,
       2028.61837123, 2460.80713775])

modelo basándose en RECM.

In [118]:
import numpy as np
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [119]:
r2_train = r2_score(y_train, train_preds)
r2_test = r2_score(y_test, test_preds)
print(r2_train)
print(r2_test)

0.5615551419174629
0.5671041872384911


In [120]:
mse_train = mean_squared_error(y_train, train_preds)
mse_test = mean_squared_error(y_test, test_preds)

In [121]:
rmse_train = np.sqrt(mean_squared_error(y_train, train_preds))
rmse_test = np.sqrt(mean_squared_error(y_test, test_preds))
print(rmse_train)
print(rmse_test)

1139.1040937388918
1092.8630817241497


2. modelo de árbol de regresión para predecir las ventas.

modelo simple de árbol de regresión.

rendimiento del modelo basándose en r^2.

rendimiento del modelo basándose en RECM.  

In [122]:
from sklearn.tree import DecisionTreeRegressor

In [123]:
# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [124]:
#insanciar modelo 
dec_tree = DecisionTreeRegressor(random_state = 42)

In [125]:
dec_tree.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [128]:
train_preds = dec_tree.predict(X_train)
test_preds = dec_tree.predict(X_test)

In [129]:
# evalúen el modelo
train_score = dec_tree.score(X_train, y_train)
test_score = dec_tree.score(X_test, y_test)
print(train_score)
print(test_score)

1.0
0.18478787410509645


In [130]:
#ajustar este modelo
dec_tree.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 42,
 'splitter': 'best'}

In [131]:
#profundidad de nuestro árbol por defecto
dec_tree.get_depth()

40

In [132]:
#nodos hojas

dec_tree.get_n_leaves()

6265

In [133]:
#fijemos nuestra profundidad máxima en 3
dec_tree_2 = DecisionTreeRegressor(max_depth = 3, random_state = 42)
dec_tree_2.fit(X_train, y_train)
train_2_score = dec_tree_2.score(X_train, y_train)
test_2_score = dec_tree_2.score(X_test, y_test)
print(train_2_score)
print(test_2_score)

0.5242182307911816
0.5242218617296541


In [140]:
dec_tree_15 = DecisionTreeRegressor(max_depth = 8, random_state = 42)
dec_tree_15.fit(X_train, y_train)
train_15_score = dec_tree_15.score(X_train, y_train)
test_15_score = dec_tree_15.score(X_test, y_test)
print(train_15_score)
print(test_15_score)

0.6438431076054045
0.5596238106210558


Predice mejor el de arboles de decisión 